In [1]:
import numpy as np
import os
from scipy.io import wavfile
import librosa
import csv
import math as m
from natsort import natsorted
import h5py
#in this section the features for mixed voices are extracted.
# orig_path=os.getcwd()
i = 1
Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
write_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff/10hdata'
mixed_folder = os.path.normpath(os.path.join(write_path,'mixed_10h'))
# f.close()
a = os.listdir(os.path.join(mixed_folder))
a = natsorted(a)
mixed_logname = 'mixed_log_10h_nopadding'
mixed_phasename = 'mixed_phase_10h_nopadding'
mixed_logpath = os.path.normpath(os.path.join(write_path,mixed_logname))
mixed_phasepath = os.path.normpath(os.path.join(write_path,mixed_phasename))

f = h5py.File(mixed_logpath+'.hdf5', 'w')
d = f.create_dataset(mixed_logname, (0,257),maxshape=(None,257), dtype='f', chunks=True)
# ff.close()
ff = h5py.File(mixed_phasepath+'.hdf5', 'w')
dd = ff.create_dataset(mixed_phasename, (0,257),maxshape=(None,257), dtype='f', chunks=True)
#################################
w=3
#################################
for filename in a:
    wav_data, sr = librosa.load(os.path.join(mixed_folder,filename), sr=16000) 
    framed_data=librosa.core.stft(wav_data, n_fft=512, hop_length=256, win_length=512, window='hann')
#     print(framed_data.shape)
    abslt=np.absolute(framed_data)**2
    dft_signal=np.log10(abslt+1e-7*np.ones(np.shape(abslt)))
    for i in range(m.floor(w/2)):
        dft_signal=np.insert(dft_signal,0,0,axis = 1)
    data_phase=np.angle(framed_data)
    for i in range(m.floor(w/2)):
        data_phase=np.insert(data_phase,0,0,axis = 1)
    d.resize(d.shape[0]+dft_signal.shape[1], axis=0)   
    d[-1*dft_signal.shape[1]:] = dft_signal.T
    dd.resize(dd.shape[0]+data_phase.shape[1], axis=0)   
    dd[-1*data_phase.shape[1]:] = data_phase.T
    i=i+1
####################################################
####################################################
import h5py
import math as m
import os
import numpy as np
import librosa
from natsort import natsorted
orig_path=os.getcwd()
wav=[]
clean_raw = 'clean_10h'
# l=os.listdir(os.path.normpath(os.path.join(Data_path,clean_raw)))
# l=natsorted(l)
# filename1 = l[1]
i=0
###################################################
w=3
###################################################
num_of_snrs=6
num_of_speakers=2
files = os.listdir(os.path.join(write_path,clean_raw))
files=natsorted(files)
# f.close()
# ff.close()
clean_log = 'single_dataset_log_10h_nopadding'
clean_phase = 'single_dataset_phase_10h_nopadding'
clean_log_path = os.path.normpath(os.path.join(write_path,clean_log))
clean_phase_path = os.path.normpath(os.path.join(write_path,clean_phase))
f = h5py.File(clean_log_path+'.hdf5', 'w')
d = f.create_dataset(clean_log, (0,257),maxshape=(None,257), dtype='f', chunks=True)
ff = h5py.File(clean_phase_path+'.hdf5', 'w')
dd = ff.create_dataset(clean_phase, (0,257),maxshape=(None,257), dtype='f', chunks=True)
#in this section the features for clean voices are extracted.
for voice in files:
    wav_data, sr = librosa.load(os.path.join(write_path,clean_raw,voice), sr=16000) 
    framed_data=librosa.core.stft(wav_data, n_fft=512, hop_length=256, win_length=512, window='hann')
#     print(framed_data.shape)
    abslt=np.absolute(framed_data)**2
    dft_signal=np.log10(abslt+1e-7*np.ones(np.shape(abslt)))
    data_phase=np.angle(framed_data)
#     for i in range(m.floor(w/2)):
#         dft_signal=np.insert(dft_signal,0,0,axis = 1)
#     for i in range(num_of_snrs*num_of_speakers):
    d.resize(d.shape[0]+dft_signal.shape[1], axis=0)   
    d[-1*dft_signal.shape[1]:] = dft_signal.T
    dd.resize(dd.shape[0]+data_phase.shape[1], axis=0)   
    dd[-1*data_phase.shape[1]:] = data_phase.T
f.close()
ff.close()
###############################################
###############################################
import math as m
import numpy as np
import h5py
##############################################
w = 3
###############################################
mixed_logname='mixed_log_10h_nopadding'
mixed_phasename='mixed_phase_10h_nopadding'
mixed_log_path = os.path.normpath(os.path.join(write_path,mixed_logname))
clean_phase_path = os.path.normpath(os.path.join(write_path,mixed_phasename))
import h5py
h5f = h5py.File(mixed_log_path+'.hdf5','r')
ftr = h5f[mixed_logname][0:]
# h5f.close()
ftr_refrmd=[]
print('0')
for i in range(m.floor(w/2)):
    p=-m.floor(w/2)
    temp = []
    for j in range(w):
        if np.all(ftr[i]==0):
            break
        temp.extend(ftr[i+p+j])
    if np.any(temp!=[]):
        ftr_refrmd.append(temp)
f.close()
print('1')
refrmd_file='ftr_refrmd_10h_nopadding'
refrmd_file_path = os.path.normpath(os.path.join(write_path,refrmd_file))
f = h5py.File(refrmd_file_path+'.hdf5', 'w')
d = f.create_dataset(refrmd_file, (0,257*w),maxshape=(None,257*w), dtype='f', chunks=True)
print('2')
for i in range(m.floor(w/2),ftr.shape[0]-m.floor(w/2)-1):
    k=-m.floor(w/2)
    temp = []
    for j in range(w):
        if np.all(ftr[i]==0):
            break
        temp.extend(ftr[i+k])
        k=k+1
    if np.any(temp!=[]):
        ftr_refrmd.append(temp)
    if len(ftr_refrmd)>50000 or i==ftr.shape[0]-m.floor(w/2)-2:
        d.resize(d.shape[0]+len(ftr_refrmd), axis=0)   
        d[-1*len(ftr_refrmd):] = ftr_refrmd
        ftr_refrmd=[]
    if i%1000==0:
        print(i)
for i in range(ftr.shape[0]-m.floor(w/2)-1,ftr.shape[0]):
    p=-m.floor(w/2)
    temp = []
    for j in range(w):
        if np.all(ftr[i]==0):
            break
        if j>m.floor(w/2)+(ftr.shape[0]-i)-1:
            temp.extend(np.zeros(ftr[ftr.shape[0]-1].shape))
        else:
            temp.extend(ftr[i+p+j])
    if np.any(temp!=[]):
        ftr_refrmd.append(temp)
d.resize(d.shape[0]+len(ftr_refrmd), axis=0)   
d[-1*len(ftr_refrmd):] = ftr_refrmd
ftr_refrmd=[]
f.close()
h5f.close()
##############################################
##############################################
# import h5py
# import numpy as np
# h5 = h5py.File(refrmd_file, 'r')
# d=h5[refrmd_file]
# tot_len=d.shape[0]
# lst=0
# h5.close()
# on=0
# h5f = h5py.File(refrmd_file+'.hdf5','r')
# for i in range(0,tot_len,1000):
#     mixed = h5f[refrmd_file][i:i+1000]
#     lst=lst+np.sum(mixed,axis=0)
#     on=on+len(mixed)
# tot_mean=lst/tot_len
# h5f.close()
# lst=0
# h5f = h5py.File(refrmd_file+'.hdf5','r')
# for i in range(0,tot_len,1000):
#     mixed = h5f[refrmd_file][i:i+1000]
#     lst=lst+np.sum((mixed - tot_mean)**2,axis=0)
# lst=lst/tot_len
# tot_std=np.sqrt(lst)
# h5f.close()
# f = h5py.File('ftr_scaled.hdf5', 'w')
# dd = f.create_dataset('ftr_scaled', (0,1799),maxshape=(None,1799), dtype='f', chunks=True)
# h5f = h5py.File('ftr_refrmd.hdf5','r')
# for i in range(0,tot_len,1000):
#     mixed = h5f['ftr_refrmd'][i:i+1000]
#     scaled=(mixed-tot_mean)/tot_std
#     dd.resize(dd.shape[0]+scaled.shape[0], axis=0)   
# np.savetxt('mean.txt',tot_mean)
# np.savetxt('std.txt',tot_std)

0
1
2
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158

1165000
1166000
1167000
1168000
1169000
1170000
1171000
1172000
1173000
1174000
1175000
1176000
1177000
1178000
1179000
1180000
1181000
1182000
1183000
1184000
1185000
1186000
1187000
1188000
1189000
1190000
1191000
1192000
1193000
1194000
1195000
1196000
1197000
1198000
1199000
1200000
1201000
1202000
1203000
1204000
1205000
1206000
1207000
1208000
1209000
1210000
1211000
1212000
1213000
1214000
1215000
1216000
1217000
1218000
1219000
1220000
1221000
1222000
1223000
1224000
1225000
1226000
1227000
1228000
1229000
1230000
1231000
1232000
1233000
1234000
1235000
1236000
1237000
1238000
1239000
1240000
1241000
1242000
1243000
1244000
1245000
1246000
1247000
1248000
1249000
1250000
1251000
1252000
1253000
1254000
1255000
1256000
1257000
1258000
1259000
1260000
1261000
1262000
1263000
1264000
1265000
1266000
1267000
1268000
1269000
1270000
1271000
1272000
1273000
1274000
1275000
1276000
1277000
1278000
1279000
1280000
1281000
1282000
1283000
1284000
1285000
1286000
1287000
1288000
1289000


2190000
2191000
2192000
2193000
2194000
2195000
2196000
2197000
2198000
2199000
2200000
2201000
2202000
2203000
2204000
2205000
2206000
2207000
2208000
2209000
2210000
2211000
2212000
2213000
2214000
2215000


In [ ]:
h5 = h5py.File('ftr_refrmd_10h', 'r')
d=h5[refrmd_file]

In [4]:
import numpy as np
import tensorflow as tf
import os
__author__ = "Sangwoong Yoon"

def np_to_tfrecords(X, Y, file_path_prefix, verbose=True):
    """
    Converts a Numpy array (or two Numpy arrays) into a tfrecord file.
    For supervised learning, feed training inputs to X and training labels to Y.
    For unsupervised learning, only feed training inputs to X, and feed None to Y.
    The length of the first dimensions of X and Y should be the number of samples.
    
    Parameters
    ----------
    X : numpy.ndarray of rank 2
        Numpy array for training inputs. Its dtype should be float32, float64, or int64.
        If X has a higher rank, it should be rshape before fed to this function.
    Y : numpy.ndarray of rank 2 or None
        Numpy array for training labels. Its dtype should be float32, float64, or int64.
        None if there is no label array.
    file_path_prefix : str
        The path and name of the resulting tfrecord file to be generated, without '.tfrecords'
    verbose : bool
        If true, progress is reported.
    
    Raises
    ------
    ValueError
        If input type is not float (64 or 32) or int.
    
    """
    def _dtype_feature(ndarray):
        """match appropriate tf.train.Feature class with dtype of ndarray. """
        assert isinstance(ndarray, np.ndarray)
        dtype_ = ndarray.dtype
        if dtype_ == np.float64 or dtype_ == np.float32:
            return lambda array: tf.train.Feature(float_list=tf.train.FloatList(value=array))
        elif dtype_ == np.int64:
            return lambda array: tf.train.Feature(int64_list=tf.train.Int64List(value=array))
        else:  
            raise ValueError("The input should be numpy ndarray. \
                               Instaed got {}".format(ndarray.dtype))
            
    assert isinstance(X, np.ndarray)
    assert len(X.shape) == 2  # If X has a higher rank, 
                               # it should be rshape before fed to this function.
    assert isinstance(Y, np.ndarray) or Y is None
    
    # load appropriate tf.train.Feature class depending on dtype
    dtype_feature_x = _dtype_feature(X)
    if Y is not None:
        assert X.shape[0] == Y.shape[0]
        assert len(Y.shape) == 2
        dtype_feature_y = _dtype_feature(Y)            
    
    # Generate tfrecord writer
    result_tf_file = file_path_prefix + '.tfrecords'
    writer = tf.python_io.TFRecordWriter(result_tf_file)
    if verbose:
        print("Serializing {:d} examples into {}".format(X.shape[0], result_tf_file))
        
    # iterate over each sample,
    # and serialize it as ProtoBuf.
    for idx in range(X.shape[0]):
        x = X[idx]
        if Y is not None:
            y = Y[idx]
        
        d_feature = {}
        d_feature['X'] = dtype_feature_x(x)
        if Y is not None:
            d_feature['Y'] = dtype_feature_y(y)
            
        features = tf.train.Features(feature=d_feature)
        example = tf.train.Example(features=features)
        serialized = example.SerializeToString()
        writer.write(serialized)
    
    if verbose:
        print("Writing {} done!".format(result_tf_file))

        
#################################    
##      Test and Use Cases     ##
#################################
import h5py
load_size = 100000
k = 0
Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
write_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff/10hdata'
input_name = 'ftr_refrmd_10h_nopadding'
input_path = os.path.normpath(os.path.join(write_path,input_name))
target_name = 'single_dataset_log_10h_nopadding'
target_path = os.path.normpath(os.path.join(write_path,target_name))
h5f1 = h5py.File(input_path+'.hdf5','r')
hh = h5py.File(target_path+'.hdf5', 'r')
d=hh[target_name]
data_len = d.shape[0]
index = np.arange(0, data_len, load_size)
for I in index:
    if data_len-I<load_size:
        xx = h5f1[input_name][I:]
        yy = hh[target_name][I:]
    else:
        xx = h5f1[input_name][I:I+load_size]
        yy = hh[target_name][I:I+load_size]
    np_to_tfrecords(xx, yy, os.path.normpath(os.path.join(Data_path,'tfrecord_files','tfrecord_files_10h_nopadding','filenum'+str(k))), verbose=True)
    k = k+1
# 1-2. Check if the data is stored correctly
# open the saved file and check the first entries
# for serialized_example in tf.python_io.tf_record_iterator('test.tfrecords'):
#     example = tf.train.Example()
#     example.ParseFromString(serialized_example)
#     x_1 = np.array(example.features.feature['X'].float_list.value)
#     y_1 = np.array(example.features.feature['Y'].float_list.value)
#     break

# np_to_tfrecords(X, Y, file_path_prefix, verbose=True)

Serializing 100000 examples into D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\tfrecord_files_10h_nopadding\filenum0.tfrecords
Writing D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\tfrecord_files_10h_nopadding\filenum0.tfrecords done!
Serializing 100000 examples into D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\tfrecord_files_10h_nopadding\filenum1.tfrecords
Writing D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\tfrecord_files_10h_nopadding\filenum1.tfrecords done!
Serializing 100000 examples into D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\tfrecord_files_10h_nopadding\filenum2.tfrecords
Writing D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\tfrecord_files_10h_nopadding\filenum2.tfrecords done!
Serializing 100000 examples into D:\studies\unive

In [10]:
import os
import numpy as np
import tensorflow as tf

Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'

def decode(serialized_example):
  # NOTE: You might get an error here, because it seems unlikely that the features
  # called 'coord2d' and 'coord3d', and produced using `ndarray.flatten()`, will
  # have a scalar shape. You might need to change the shape passed to
  # `tf.FixedLenFeature()`.
  features = tf.parse_single_example(
      serialized_example,
      features={'X': tf.FixedLenFeature([], tf.float32),
                'Y': tf.FixedLenFeature([], tf.float32)})

  # NOTE: No need to cast these features, as they are already `tf.float32` values.
  return features['X'], features['Y']

filename = [os.path.normpath(os.path.join(Data_path,'tfrecord_files','tfrecord_files_10h','filenum1.tfrecords'))]
dataset = tf.data.TFRecordDataset(filename).map(decode)
iterator = dataset.make_one_shot_iterator()
c2d, c3d = iterator.get_next()

with tf.Session() as sess:

  try:

    while True:
      print(sess.run((c2d, c3d)))

  except tf.errors.OutOfRangeError:
    # Raised when we reach the end of the file.
    pass

InvalidArgumentError: Key: Y.  Can't parse serialized Example.
	 [[{{node ParseSingleExample/ParseSingleExample}}]]
	 [[node IteratorGetNext_5 (defined at <ipython-input-10-b34ddd7ec8a2>:23) ]]

In [49]:
input_name = 'ftr_refrmd_10h'
write_path2 = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff/10hdata'
input_path = os.path.normpath(os.path.join(write_path2,input_name))
h5f1 = h5py.File(input_path+'.hdf5','r')
xx = h5f1[input_name][512000]

In [50]:
xx

array([ 1.3545911 ,  1.0818698 , -0.24872012,  1.9945492 ,  2.1653678 ,
        1.3879275 ,  1.5467528 ,  2.3365054 ,  1.7024057 , -0.24777055,
        1.2678083 ,  1.4279422 ,  0.3440746 ,  0.97298187,  1.4189924 ,
        0.41704223,  1.2393667 ,  0.8722655 ,  0.05412256, -0.51407814,
       -0.1445315 , -0.63822216, -0.48826015, -1.145613  , -0.24516976,
       -0.20487943, -0.6513603 , -0.38496923,  0.01301329, -0.9847468 ,
       -0.84937036, -0.5106788 , -0.8337593 , -1.246424  , -1.443633  ,
       -3.3127267 , -1.4137309 , -2.3016465 , -1.1688585 , -1.0775933 ,
       -1.6349958 , -1.8429928 , -1.5689799 , -1.5359442 , -0.76302797,
       -0.3062101 , -0.84923464, -0.7657294 , -0.59773594, -0.8394146 ,
       -2.0309086 , -2.1918364 , -1.3592702 , -3.263354  , -1.2084008 ,
       -0.82110095, -0.82541966, -1.3016543 , -1.4735366 , -1.4991721 ,
       -1.3220024 , -1.4699916 , -1.4342308 , -1.985925  , -1.400954  ,
       -1.9582542 , -1.6783314 , -1.2899138 , -1.8944085 , -1.20

In [42]:
xx[0][256:]

array([ 0.        ,  0.95226926,  0.81424624,  0.26638246, -1.0870386 ,
       -3.2679992 , -3.6111832 , -3.39885   , -5.034203  , -4.082465  ,
       -2.847904  , -2.7488606 , -4.2329416 , -3.5763164 , -3.866974  ,
       -4.5519357 , -3.4181383 , -3.5218778 , -4.080829  , -4.0534    ,
       -4.0750456 , -4.2322617 , -6.3595257 , -4.402423  , -4.782611  ,
       -4.86097   , -4.9555197 , -4.309063  , -3.9171703 , -4.7101007 ,
       -5.0920296 , -5.3481746 , -6.815517  , -5.0398507 , -4.3151464 ,
       -6.856251  , -4.3536572 , -5.0648646 , -5.0676265 , -4.6430793 ,
       -4.9820805 , -4.2858214 , -6.9914513 , -5.288295  , -4.5299664 ,
       -4.847857  , -4.853148  , -5.183619  , -5.1291    , -6.3025794 ,
       -4.9681296 , -5.633843  , -4.860765  , -5.718714  , -6.600328  ,
       -4.867722  , -5.9522014 , -4.4948254 , -4.092798  , -4.4694815 ,
       -5.3421984 , -6.9439893 , -6.4906354 , -5.739023  , -4.5175543 ,
       -4.463977  , -6.8780828 , -5.363199  , -5.0022388 , -4.17